# Generador datos sintéticos para el modelo Clasificador

In [ ]:
!pip install pandas openai tqdm

In [ ]:
categories = [
    {
        "id": "0",
        "category": "Infraestructura y Recursos",
        "description": """
Esta categoría abarca todas las decisiones formales relacionadas con los bienes físicos, equipamiento y recursos materiales de la institución. Las resoluciones deben ser actos administrativos que autorizan, planifican y ejecutan la gestión del patrimonio institucional. Asegúrate de que los 50 ejemplos cubran de manera diversa los siguientes subtemas:
*Obras y Mantenimiento: Autorización para el inicio de proyectos de construcción, remodelación, ampliación, o mantenimiento mayor de instalaciones (aulas, laboratorios, oficinas, áreas comunes, techos, fachadas, sistemas eléctricos).
*Adquisición de Bienes: Aprobación para la compra de equipamiento, tecnología y mobiliario. Ejemplos: computadoras, proyectores, licencias de software especializado, maquinaria para talleres, equipos de laboratorio, mobiliario para aulas, libros para la biblioteca.
*Uso y Asignación de Espacios Físicos: Definición y oficialización de la asignación de espacios. Ejemplos: 'Se asigna el laboratorio de química L-203 para uso exclusivo de la carrera de Ingeniería Agroindustrial' o 'Se autoriza el uso del auditorio principal para las jornadas académicas de la carrera de Software los días 15 y 16 de mayo'.
*Baja de Activos: Autorización para dar de baja bienes obsoletos, dañados o en desuso, siguiendo el procedimiento formal. Ejemplo: 'Aprobar la baja de 25 monitores CRT del inventario del laboratorio de computación 3'.
*Adjudicación de Contratos y Licitaciones: Formalización de la selección de proveedores para ejecutar obras, prestar servicios de mantenimiento o suministrar materiales. Ejemplo: 'Adjudicar el contrato para el mantenimiento preventivo de los sistemas de aire acondicionado a la empresa 'ClimaConfort S.A.'.
*Aceptación de Donaciones: Autorización formal para aceptar donaciones de equipos, materiales o infraestructura por parte de terceros (empresas, exalumnos, fundaciones).
"""
    }, {
        "id": "1",
        "category": "Nombramientos de Personal",
        "description": """
Esta categoría se refiere a todas las decisiones formales relacionadas con la gestión humana del instituto. Las resoluciones son el instrumento legal que vincula a la persona con la institución, definiendo su rol, derechos y obligaciones.
Temas que incluiría:
Contratación y Nombramiento: Designar efectivamente a profesores, instructores, investigadores, personal administrativo y de servicios. Incluye el cargo, la dedicación (tiempo completo, parcial), la asignatura o área, y la duración del contrato.
Encargos de Cargo: Asignar temporalmente a una persona las funciones de un cargo vacante o de una persona con licencia (ej: "Se encarga la Jefatura de Carrera de Informática al Prof. Juan Pérez").
Renuncias y Ceses: Aceptar formalmente la renuncia de un miembro del personal o dar por terminada una relación laboral.
Licencias y Permisos: Autorizar licencias del personal por motivos de salud, estudio, maternidad/paternidad, personales o comisión de servicio.
Ascensos y Traslados: Oficializar cambios de categoría, promociones internas o el movimiento de personal entre diferentes departamentos o sedes.
Designación de Roles Específicos: Nombrar tutores de grupo, coordinadores de área, responsables de laboratorio, jurados de examen, etc.
"""
    }, {
        "id": "2",
        "category": "Aprobaciones de Planes",
        "description": """
Esta es la columna vertebral de la actividad académica. Las resoluciones en esta categoría validan y oficializan toda la oferta educativa y los proyectos centrales del instituto, dándoles validez formal.
Temas que incluiría:
Planes y Programas de Estudio: Aprobar la creación, modificación o extinción de una carrera técnica, terciaria o de posgrado. Esto incluye validar su malla curricular, contenidos mínimos de cada materia, carga horaria y régimen de correlatividades.
Proyectos de Investigación y Desarrollo: Autorizar el inicio de proyectos de investigación, asignarles un presupuesto y reconocer a sus responsables.
Plan Anual de Actividades: Aprobar el calendario académico oficial (inicio y fin de cuatrimestres, períodos de examen, fechas de inscripción, días feriados institucionales).
Planes de Capacitación: Validar programas de formación continua para el personal docente y administrativo.
Presupuesto Anual: Aunque tiene un componente económico, su aprobación formal es un acto de gobierno que se oficializa mediante resolución, dándole validez para ejecutar el gasto.
"""
    },{
        "id": "3",
        "category": "Modificaciones Reglamentarias",
        "description": """
Esta categoría es el marco legal interno del instituto. Las resoluciones aquí se utilizan para crear, cambiar o derogar las "reglas del juego" que rigen el comportamiento y los procesos de toda la comunidad educativa.
Temas que incluiría:
Creación o Reforma de Reglamentos: Elaborar y aprobar documentos como:
Reglamento de Enseñanza y Evaluación.
Reglamento de Conducta y Régimen Disciplinario (para estudiantes y/o personal).
Reglamento de Investigación.
Reglamento de Biblioteca.
Reglamento de Pasantías y Prácticas Profesionales.
Reglamento de Uso de Laboratorios y Talleres.
Protocolos y Procedimientos: Establecer procedimientos oficiales para abordar situaciones específicas, como protocolos de actuación ante acoso escolar (bullying), accesibilidad para personas con discapacidad, o seguridad industrial en talleres.
Actualización de Aranceles y Tarifas: Modificar formalmente los costos de matrícula, cuotas, derechos de examen o otros conceptos, lo que usualmente requiere una modificación del reglamento económico.
Políticas Institucionales: Implementar políticas transversales, como políticas de equidad de género, de inclusión educativa o de sostenibilidad ambiental.
"""
    }]

In [ ]:
#Configurar API de DeepSeek con sintaxis actualizada
from openai import OpenAI
import pandas as pd
import json

client = OpenAI(
    api_key="<API-PROPIA>",
    base_url="https://api.deepseek.com/v1"
)

def callPromptLLM(category, description, id):


    prompt = f"""
Tu tarea es crear 50 ejemplos detallados de resoluciones institucionales. Todas las resoluciones deben pertenecer exclusivamente a la categoría de '{category}'. El propósito de estos datos es entrenar un modelo de lenguaje para clasificación de documentos, por lo que la coherencia, diversidad y realismo son fundamentales.

Categoría a Generar: '{category}'
Descripción de la Categoría: '{description}'

Reglas Estructurales Obligatorias
Cada una de las 50 resoluciones debe seguir estas reglas sin excepción:
*Formato de Inicio: Toda resolución debe comenzar exactamente con Artículo 1.-.
*Número de Artículos: Cada resolución puede tener entre 1 y 3 artículos en total.
*Coherencia Semántica: Los artículos 2 y 3, si existen, deben estar directamente relacionados con el Artículo 1. Deben ser una consecuencia lógica, una delegación de responsabilidad, una aclaración o una disposición complementaria.
  Ejemplo de coherencia: Si el Artículo 1 aprueba la compra de equipos, el Artículo 2 debe designar al responsable de supervisar la compra o al departamento que la ejecutará, y el Artículo 3 podría establecer el plazo para dicha adquisición.
  Ejemplo de incoherencia (a evitar): Si el Artículo 1 aprueba una obra de mantenimiento, el Artículo 2 no puede tratar sobre la aprobación de un nuevo reglamento académico.
*Longitud y Tono:
  Cada artículo debe ser un párrafo corto, de entre 15 y 100 palabras.
  El lenguaje debe ser formal, impersonal y preciso, propio de un documento administrativo. Evita cualquier tipo de lenguaje coloquial.
*Diversidad:
  Utiliza diferentes nombres de personas, cargos (Rector, Coordinador Administrativo, Director de Planificación) y departamentos (Unidad de Compras Públicas, Departamento de Mantenimiento, Coordinación de Laboratorios).
  Asegúrate de que los 50 ejemplos sean distintos entre sí en cuanto al bien, servicio u obra que se resuelve.
*Ejemplos de Estructura Válida
  Ejemplo A (2 Artículos):
  Artículo 1.- Aprobar la adquisición e instalación de diez (10) proyectores multimedia de alta definición para las aulas del bloque B del Instituto Superior Tecnológico Central, con cargo a la partida presupuestaria 730104.
  Artículo 2.- Designar a la Dirección Administrativa y Financiera como la unidad encargada de ejecutar el proceso de contratación pública correspondiente, en un plazo no mayor a 45 días a partir de la emisión de la presente resolución.

  Ejemplo B (1 Artículo):
  Artículo 1.- Autorizar el inicio del proceso de mantenimiento preventivo y correctivo de la infraestructura eléctrica del laboratorio de manufactura de la Escuela Politécnica Nacional, con el objetivo de garantizar la seguridad y operatividad de los equipos para el próximo período académico.

  Ejemplo C (3 Artículos):
  Artículo 1.- Aprobar la readecuación del espacio físico anteriormente ocupado por la bodega general para la implementación de una nueva sala de lectura y estudio para los estudiantes de la Universidad Técnica del Norte.
  Artículo 2.- Disponer que el Departamento de Planificación y Obras elabore y presente el diseño técnico y el presupuesto detallado del proyecto en un plazo máximo de 30 días.
  Artículo 3.- Encargar a la Coordinación de Bienestar Estudiantil la supervisión del proyecto una vez que este sea adjudicado, asegurando que el espacio cumpla con las necesidades del estudiantado.
*Formato de salida
  va a ser un texto que represente un csv, el cual va a tener 2 columnas
  id|resolucion
  para este caso coloca id= {id} en todos los registros
  usa el caracter PIPE | como separador para evitar conflictos con comas en el texto, sin embargo traeme los datos sin cabecera, solo los registros

Ejemplo de formato:
{id}|Artículo 1.- Aprobar la compra de equipos...
{id}|Artículo 1.- Designar al Ing. Juan Pérez...
Inicia la generación de los 50 ejemplos ahora, siguiendo estrictamente todas las pautas mencionadas.
                """
    print(category, description, id)
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content":"""
                  Actúa como un generador experto de documentos administrativos para instituciones de educación superior.
                """},
                {"role": "user", "content": prompt}
            ],
            max_tokens=5000,
            temperature=0.5,
            top_p=0.9,
            frequency_penalty=0.3,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error procesando")
        print(e)
        return None

In [ ]:
import concurrent.futures
from typing import List
from tqdm import tqdm

# Configuración de ejecución
num_threads = 2  # Puedes cambiar este valor según necesites
num_categories = len(categories)

all_results = []
total_tasks = num_threads * num_categories

with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Crear lista de tareas (cada categoría se ejecutará num_threads veces)
    futures = []
    for _ in range(num_threads):
        for i in range(num_categories):
            cat = categories[i]
            futures.append(
                executor.submit(
                    callPromptLLM,
                    cat.get("category"),
                    cat.get("description"),
                    cat.get("id")
                )
            )

    # Monitorear progreso con barra
    with tqdm(total=total_tasks, desc="Procesando tareas") as pbar:
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                if result is not None:
                    all_results.append(result)
                pbar.update(1)
            except Exception as e:
                print(f"\nError: {e}")

print(f"\nProceso completado. Total resultados: {len(all_results)}")

Infraestructura y Recursos 
Esta categoría abarca todas las decisiones formales relacionadas con los bienes físicos, equipamiento y recursos materiales de la institución. Las resoluciones deben ser actos administrativos que autorizan, planifican y ejecutan la gestión del patrimonio institucional. Asegúrate de que los 50 ejemplos cubran de manera diversa los siguientes subtemas:
*Obras y Mantenimiento: Autorización para el inicio de proyectos de construcción, remodelación, ampliación, o mantenimiento mayor de instalaciones (aulas, laboratorios, oficinas, áreas comunes, techos, fachadas, sistemas eléctricos).
*Adquisición de Bienes: Aprobación para la compra de equipamiento, tecnología y mobiliario. Ejemplos: computadoras, proyectores, licencias de software especializado, maquinaria para talleres, equipos de laboratorio, mobiliario para aulas, libros para la biblioteca.
*Uso y Asignación de Espacios Físicos: Definición y oficialización de la asignación de espacios. Ejemplos: 'Se asigna e

Procesando tareas:  12%|█▎        | 1/8 [01:08<07:56, 68.04s/it]

Aprobaciones de Planes 
Esta es la columna vertebral de la actividad académica. Las resoluciones en esta categoría validan y oficializan toda la oferta educativa y los proyectos centrales del instituto, dándoles validez formal.
Temas que incluiría:
Planes y Programas de Estudio: Aprobar la creación, modificación o extinción de una carrera técnica, terciaria o de posgrado. Esto incluye validar su malla curricular, contenidos mínimos de cada materia, carga horaria y régimen de correlatividades.
Proyectos de Investigación y Desarrollo: Autorizar el inicio de proyectos de investigación, asignarles un presupuesto y reconocer a sus responsables.
Plan Anual de Actividades: Aprobar el calendario académico oficial (inicio y fin de cuatrimestres, períodos de examen, fechas de inscripción, días feriados institucionales).
Planes de Capacitación: Validar programas de formación continua para el personal docente y administrativo.
Presupuesto Anual: Aunque tiene un componente económico, su aprobación 

Procesando tareas:  25%|██▌       | 2/8 [01:54<05:31, 55.27s/it]

Modificaciones Reglamentarias 
Esta categoría es el marco legal interno del instituto. Las resoluciones aquí se utilizan para crear, cambiar o derogar las "reglas del juego" que rigen el comportamiento y los procesos de toda la comunidad educativa.
Temas que incluiría:
Creación o Reforma de Reglamentos: Elaborar y aprobar documentos como:
Reglamento de Enseñanza y Evaluación.
Reglamento de Conducta y Régimen Disciplinario (para estudiantes y/o personal).
Reglamento de Investigación.
Reglamento de Biblioteca.
Reglamento de Pasantías y Prácticas Profesionales.
Reglamento de Uso de Laboratorios y Talleres.
Protocolos y Procedimientos: Establecer procedimientos oficiales para abordar situaciones específicas, como protocolos de actuación ante acoso escolar (bullying), accesibilidad para personas con discapacidad, o seguridad industrial en talleres.
Actualización de Aranceles y Tarifas: Modificar formalmente los costos de matrícula, cuotas, derechos de examen o otros conceptos, lo que usualm

...

Aprobaciones de Planes 
Esta es la columna vertebral de la actividad académica. Las resoluciones en esta categoría validan y oficializan toda la oferta educativa y los proyectos centrales del instituto, dándoles validez formal.
Temas que incluiría:
Planes y Programas de Estudio: Aprobar la creación, modificación o extinción de una carrera técnica, terciaria o de posgrado. Esto incluye validar su malla curricular, contenidos mínimos de cada materia, carga horaria y régimen de correlatividades.
Proyectos de Investigación y Desarrollo: Autorizar el inicio de proyectos de investigación, asignarles un presupuesto y reconocer a sus responsables.
Plan Anual de Actividades: Aprobar el calendario académico oficial (inicio y fin de cuatrimestres, períodos de examen, fechas de inscripción, días feriados institucionales).
Planes de Capacitación: Validar programas de formación continua para el personal docente y administrativo.
Presupuesto Anual: Aunque tiene un componente económico, su aprobación 

Procesando tareas:  75%|███████▌  | 6/8 [04:12<01:12, 36.29s/it]

Modificaciones Reglamentarias 
Esta categoría es el marco legal interno del instituto. Las resoluciones aquí se utilizan para crear, cambiar o derogar las "reglas del juego" que rigen el comportamiento y los procesos de toda la comunidad educativa.
Temas que incluiría:
Creación o Reforma de Reglamentos: Elaborar y aprobar documentos como:
Reglamento de Enseñanza y Evaluación.
Reglamento de Conducta y Régimen Disciplinario (para estudiantes y/o personal).
Reglamento de Investigación.
Reglamento de Biblioteca.
Reglamento de Pasantías y Prácticas Profesionales.
Reglamento de Uso de Laboratorios y Talleres.
Protocolos y Procedimientos: Establecer procedimientos oficiales para abordar situaciones específicas, como protocolos de actuación ante acoso escolar (bullying), accesibilidad para personas con discapacidad, o seguridad industrial en talleres.
Actualización de Aranceles y Tarifas: Modificar formalmente los costos de matrícula, cuotas, derechos de examen o otros conceptos, lo que usualm

Procesando tareas: 100%|██████████| 8/8 [05:50<00:00, 43.82s/it]


Proceso completado. Total resultados: 8


In [ ]:
import pandas as pd

def process_results_no_header(all_results):
    """
    Procesa resultados que vienen sin cabecera, solo con datos en formato id|resolucion
    """
    all_rows = []

    for i, csv_string in enumerate(all_results):
        try:
            lines = csv_string.strip().split('\n')

            for line in lines:
                line = line.strip()
                # Buscar líneas con formato: id|texto
                if '|' in line:
                    parts = line.split('|', 1)  # Dividir solo en el primer pipe
                    if len(parts) == 2:
                        id_part = parts[0].strip()
                        text_part = parts[1].strip()

                        # Validar que sea una fila de datos válida
                        if id_part.isdigit() and text_part.startswith('Artículo'):
                            all_rows.append({
                                'id': id_part,
                                'resolucion': text_part
                            })

            print(f"Procesado chunk {i+1} con {len([l for l in lines if '|' in l and l.split('|')[0].strip().isdigit()])} filas válidas")

        except Exception as e:
            print(f"Error procesando chunk {i+1}: {e}")
            print(f"Línea problemática: {line if 'line' in locals() else 'N/A'}")

    if all_rows:
        final_df = pd.DataFrame(all_rows)
        # Convertir y limpiar IDs
        final_df['id'] = pd.to_numeric(final_df['id'], errors='coerce')
        final_df = final_df.dropna(subset=['id', 'resolucion'])
        final_df['id'] = final_df['id'].astype(int)

        print(f"\nDataFrame final creado con {len(final_df)} filas")
        return final_df
    else:
        print("No se encontraron datos válidos")
        return pd.DataFrame()

# Usar la función
df = process_results_no_header(all_results)

Procesado chunk 1 con 29 filas válidas
Procesado chunk 2 con 24 filas válidas
Procesado chunk 3 con 30 filas válidas
Procesado chunk 4 con 36 filas válidas
Procesado chunk 5 con 30 filas válidas
Procesado chunk 6 con 23 filas válidas
Procesado chunk 7 con 41 filas válidas
Procesado chunk 8 con 30 filas válidas

DataFrame final creado con 209 filas


In [ ]:
df

,id,resolucion
0,1,Artículo 1.- Designar a la Dra. María Fernanda...
1,1,Artículo 1.- Encargar temporalmente la Jefatur...
2,1,Artículo 1.- Aceptar la renuncia voluntaria pr...
3,1,Artículo 1.- Otorgar licencia con goce de habe...
4,1,Artículo 1.- Ascender al Dr. Roberto Carlos Sá...
...,...,...
204,3,"Artículo 1.- Aprobar lanueva ""Políticade Movil..."
205,3,Artículo 1.- Incorporarenel Reglamentode Bibli...
206,3,Artículo 1.- Modificarel procedimientode selec...
207,3,Artículo 1.- Derogarel AcuerdoNo. 145-P-2018qu...


In [ ]:
df.to_csv('resoluciones_sinteticas.csv', index=False)
print("DataFrame guardado como 'resoluciones_sinteticas.csv'")

DataFrame guardado como 'resoluciones_sinteticas.csv'
